#Reload the function by Haolun huang and split the data by Chenghan Lu, Loading the data by Zekai Zhang

In [ ]:
import os
import sys
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow import keras


In [ ]:
print(tf.__version__)



2.12.0


In [ ]:
!pip install -U keras-cv-attention-models


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 635.8/635.8 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 591.0/591.0 kB 12.7 MB/s eta 0:00:00


In [ ]:
#import google drive into the colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#All the file location in the share google drive

filepath = '/content/drive/MyDrive/CSCI3397_Project/TBX11K' #gernal file pass
filepath_anno = filepath + '/annotations/json/all_trainval.json' #all the annotation in Jason file, include all the image in the TBX11K
filepath_img_train = filepath + '/imgs/train' #all the training Image (inlcude three category: health, sick, TB)
filepath_img_test = filepath + '/imgs/test' #all the test images (all display unknown and jason file have the correct label)

In [ ]:
#Exame all the annotations
import json
with open(filepath_anno, 'r') as f:
  annotations = json.load(f)
print(annotations['images'][0])

{'id': 1, 'file_name': 'tb/tb0003.png', 'width': 512, 'height': 512, 'date_captured': '2020-06-24 12:42:50.361310', 'license': 1, 'coco_url': '', 'flickr_url': ''}


In [ ]:
print(annotations['images'][0])

{'id': 1, 'file_name': 'tb/tb0003.png', 'width': 512, 'height': 512, 'date_captured': '2020-06-24 12:42:50.361310', 'license': 1, 'coco_url': '', 'flickr_url': ''}


In [ ]:
#TF split the train and validation data set from the all train 


train = tf.keras.utils.image_dataset_from_directory(
   filepath_img_train,
   validation_split=0.2,
   subset = 'training',
   seed=42, 
   image_size=(224, 224), #transfer the imgae size to the model required 224x224, instead of orginal 512x512
   batch_size=64)
val = tf.keras.utils.image_dataset_from_directory(
   filepath_img_train,
   validation_split=0.2,
   subset = 'validation',
   seed=42, 
   image_size=(224, 224),
   batch_size=64)

Found 8400 files belonging to 3 classes.
Using 6720 files for training.
Found 8400 files belonging to 3 classes.
Using 1680 files for validation.


In [ ]:
train #train had been change 

<_BatchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

# Code from the paper

## Pre-load the function By Zekai Zhang

In [ ]:
# Pre-load the function need for the model 
import os
import re
import datetime
import numpy as np
from itertools import groupby
from skimage import measure
from PIL import Image
from pycocotools import mask
from math import sqrt

convert = lambda text: int(text) if text.isdigit() else text.lower()
natrual_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ]

In [ ]:
#all the pre code from the paper 

convert = lambda text: int(text) if text.isdigit() else text.lower()
natrual_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ]

def resize_binary_mask(array, new_size):
    image = Image.fromarray(array.astype(np.uint8)*255)
    image = image.resize(new_size)
    return np.asarray(image).astype(np.bool_)

def close_contour(contour):
    if not np.array_equal(contour[0], contour[-1]):
        contour = np.vstack((contour, contour[0]))
    return contour

def binary_mask_to_rle(binary_mask):
    rle = {'counts': [], 'size': list(binary_mask.shape)}
    counts = rle.get('counts')
    for i, (value, elements) in enumerate(groupby(binary_mask.ravel(order='F'))):
        if i == 0 and value == 1:
                counts.append(0)
        counts.append(len(list(elements)))

    return rle

def binary_mask_to_polygon(binary_mask, tolerance=0):
    """Converts a binary mask to COCO polygon representation

    Args:
        binary_mask: a 2D binary numpy array where '1's represent the object
        tolerance: Maximum distance from original points of polygon to approximated
            polygonal chain. If tolerance is 0, the original coordinate array is returned.

    """
    polygons = []
    # pad mask to close contours of shapes which start and end at an edge
    padded_binary_mask = np.pad(binary_mask, pad_width=1, mode='constant', constant_values=0)
    contours = measure.find_contours(padded_binary_mask, 0.5)
    contours = np.subtract(contours, 1)
    for contour in contours:
        contour = close_contour(contour)
        contour = measure.approximate_polygon(contour, tolerance)
        if len(contour) < 3:
            continue
        contour = np.flip(contour, axis=1)
        segmentation = contour.ravel().tolist()
        # after padding and subtracting 1 we may get -0.5 points in our segmentation
        segmentation = [0 if i < 0 else i for i in segmentation]
        polygons.append(segmentation)

    return polygons

def create_image_info(image_id, file_name, image_size,
                      date_captured=datetime.datetime.utcnow().isoformat(' '),
                      license_id=1, coco_url="", flickr_url=""):

    image_info = {
            "id": image_id,
            "file_name": file_name,
            "width": image_size[0],
            "height": image_size[1],
            "date_captured": date_captured,
            "license": license_id,
            "coco_url": coco_url,
            "flickr_url": flickr_url
    }

    return image_info

def create_annotation_info(annotation_id, image_id, category_info, binary_mask=None,
                           image_size=None, tolerance=2, bounding_box=None):
    assert(binary_mask is not None or image_size is not None)

    if binary_mask is not None:
        if image_size is not None:
            binary_mask = resize_binary_mask(binary_mask, image_size)
        binary_mask_encoded = mask.encode(np.asfortranarray(binary_mask.astype(np.uint8)))
        area = mask.area(binary_mask_encoded)
        bounding_box = mask.toBbox(binary_mask_encoded)
    else:
        area = np.array(bounding_box[2] * bounding_box[3], dtype=int)
    if area < 20:
        print("Area of this annotation is less than 20, Skip it! image_id:", image_id, "area:", area, "bbox:", bounding_box)
        return None
    if category_info["is_crowd"]:
        is_crowd = 1
        segmentation = binary_mask_to_rle(binary_mask)
    else :
        is_crowd = 0
        if binary_mask is not None:
            binary_mask_encoded = mask.encode
            segmentation = binary_mask_to_polygon(binary_mask, tolerance)
            if not segmentation:
                return None
    if binary_mask is not None:
        annotation_info = {
            "id": annotation_id,
            "image_id": image_id,
            "category_id": category_info["id"],
            "iscrowd": is_crowd,
            "area": area.tolist(),
            "bbox": bounding_box.tolist(),
            "segmentation": segmentation,
            "width": binary_mask.shape[1],
            "height": binary_mask.shape[0],
        }
    else:
        annotation_info = {
            "id": annotation_id,
            "image_id": image_id,
            "category_id": category_info["id"],
            "iscrowd": is_crowd,
            "area": area.tolist(),
            "bbox": bounding_box.tolist(),
            "width": image_size[0],
            "height": image_size[1],
        }

    return annotation_info


In [ ]:
from keras_cv_attention_models import coatnet


#CoAtNet Model by Zekai Zhang

In [ ]:
# Only CoAtNet0 pretrained available.
mm = coatnet.CoAtNet0()

# Run prediction
from skimage.data import chelsea
imm = tf.keras.applications.imagenet_utils.preprocess_input(chelsea(), mode='torch') # Chelsea the cat
pred = mm(tf.expand_dims(tf.image.resize(imm, mm.input_shape[1:3]), 0)).numpy()
print(tf.keras.applications.imagenet_utils.decode_predictions(pred)[0])
# [('n02124075', 'Egyptian_cat', 0.99324566), ('n02123159', 'tiger_cat', 0.00381939), ... ]


93747824/93747824 [==============================] - 5s 0us/step
>>>> Load pretrained from: /root/.keras/models/coatnet0_224_imagenet.h5
35363/35363 [==============================] - 0s 0us/step
[('n02124075', 'Egyptian_cat', 0.9932486), ('n02123159', 'tiger_cat', 0.0038179806), ('n02123045', 'tabby', 0.0027284513), ('n02127052', 'lynx', 5.3533517e-05), ('n04209133', 'shower_cap', 1.1487229e-05)]


In [ ]:

tf.keras.backend.clear_session()
model = tf.keras.models.Sequential([
    tf.keras.layers.InputLayer(input_shape=[224, 224, 3]),
    mm,
    tf.keras.layers.Dropout(rate=0.4),
    tf.keras.layers.Dense(3, activation='softmax'),
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 coatnet0 (Functional)       (None, 1000)              23291218  
                                                                 
 dropout (Dropout)           (None, 1000)              0         
                                                                 
 dense (Dense)               (None, 3)                 3003      
                                                                 
Total params: 23,294,221
Trainable params: 23,282,573
Non-trainable params: 11,648
_________________________________________________________________


In [ ]:
model = coatnet.CoAtNet0(input_shape=[224, 224, 3])

>>>> Load pretrained from: /root/.keras/models/coatnet0_224_imagenet.h5


In [ ]:
optimizer = tf.keras.optimizers.SGD()

loss=tf.keras.losses.SparseCategoricalCrossentropy()

metrics = tf.keras.metrics.SparseCategoricalAccuracy(name='sparse_categorical_accuracy')

model.compile(optimizer=optimizer,
              loss=loss,
              metrics=metrics)


In [ ]:
#history = model.fit(train,
                    #epochs = 2, 
                    #batch_size = 64, 
                    #validation_data = val)
# Model wiothout Pre train is not working due to limited RAM of Google Colab

# The transfer learning Code is by All Three members of the team (Chenghan Lu Haolun Huang and Zekai Zhang)

In [ ]:
##Alternative 

#Transfer learining 


from tensorflow.keras.layers import Dense

# Freeze all layers except the last one
for layer in mm.layers[:-1]:
    layer.trainable = False

# Replace last layer with a new output layer
num_classes = 10
mm.layers[-1] = Dense(num_classes, activation='softmax')








In [ ]:
# Freeze all layers except the last one
for layer in mm.layers[:-1]:
    layer.trainable = False

In [ ]:
tf.keras.backend.clear_session()
model = tf.keras.models.Sequential([
    tf.keras.layers.InputLayer(input_shape=[224, 224, 3]),
    mm,
    tf.keras.layers.Dropout(rate=0.4),
    tf.keras.layers.Dense(3, activation='softmax'),
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 coatnet0 (Functional)       (None, 1000)              23291218  
                                                                 
 dropout (Dropout)           (None, 1000)              0         
                                                                 
 dense (Dense)               (None, 3)                 3003      
                                                                 
Total params: 23,294,221
Trainable params: 772,003
Non-trainable params: 22,522,218
_________________________________________________________________


In [ ]:
optimizer = tf.keras.optimizers.SGD()

loss=tf.keras.losses.SparseCategoricalCrossentropy()

metrics = tf.keras.metrics.SparseCategoricalAccuracy(name='sparse_categorical_accuracy')

model.compile(optimizer=optimizer,
              loss=loss,
              metrics=metrics)

In [ ]:
history = model.fit(train,
                    epochs = 2, 
                    batch_size = 64, 
                    validation_data = val)

Epoch 1/2
105/105 [==============================] - 3888s 35s/step - loss: 1.0373 - sparse_categorical_accuracy: 0.4536 - val_loss: 0.9981 - val_sparse_categorical_accuracy: 0.4518
Epoch 2/2
105/105 [==============================] - 79s 707ms/step - loss: 0.9881 - sparse_categorical_accuracy: 0.4478 - val_loss: 0.9683 - val_sparse_categorical_accuracy: 0.4518
